In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir logs/fit

## 1. Preprocesamiento de los datos

### 1.1 Preparación del entorno 

Aquí lo que hago es preaprar los imports principales y las variables de entorno. Ahora están comentadas porque como te dije en el correo estaba teniendo problemas con la versión de Cuda y Keras. Ahora lo he solucionado, pero basicamente lo que pasaba es que estaba usando una versión de Keras (Keras 3) la cual no es compatible con transfomers, al final lo soluciones activando  la variable de entorno TF_USE_LEGACY_KERAS e instalando en mi env de conda tf_keras para tener Keras 2. 

La variable TF_ENABLE_XLA la desactivé porque se supone que podía ser una causa de un problema de OOM que estaba teniendo durante el entrenamiento, pero al final resultó ser un problema con las versiones de las librerías.

La variables de entorno que tienen que ver con CUDA era porque creía que mi entorno virtual de conda (el cual al final lo he tenido que meter en un WLS2 con ubuntu porque en windows estaba teniendo problemas de compatibilidad peores, estaba cogienod) estaba cogiendo la versión de CUDA que no era, porque tenía varias instaladas. Pero era más un fallo de configuración del entorno que eso.

También he añadido una sección en la que controlo si el dispositivo con el que se va a entrenar es la GPU, pongo un creciomiento progresivo en el uso de memoria para evitar sobrecarga y también un límite para evitar de nuevo el OOM.

La línea tf.config.optimizer.set_jit(False) la usaba cuando tenía desactivado el XLA para evitar así que compilase por XLA y evitar posibles problemas de rendimiento, pero ese no era el problema. 

In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

import tensorflow as tf, gc, logging
tf.config.optimizer.set_jit(False) 
info = tf.sysconfig.get_build_info()
print("CUDA:",   info["cuda_version"])
print("cuDNN:",  info["cudnn_version"])

tf.get_logger().setLevel(logging.ERROR)

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision policy:", mixed_precision.global_policy())

#import random
import numpy as np
import transformers

# Fijamos la semilla para reproducibilidad
SEED = 42
#random.seed(SEED)
#np.random.seed(SEED)
#tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED) 
tf.config.experimental.enable_op_determinism() # Para evitar problemas de determinismo en TensorFlow 

# Hiperparámetros
NUM_LABELS = 44  # 43 emociones + 1 sin emoción

MODEL_NAME = "beomi/KcELECTRA-base" #"beomi/KcELECTRA-base" "monologg/kobert"
MAX_LENGTH = 256 # Longitud máxima de las secuencias
BATCH_SIZE = 16

DROPOUT_RATE   = 0.3      
L2_REG         = 1e-5         
UNFREEZE_EPOCH = 3        

EPOCHS = 10

BASE_LR = 1e-4
FT_LR = 2e-5
WEIGHT_DECAY   = 0.01    # Decaimiento de pesos 
BETA_1         = 0.9     # Parámetro β₁ de AdamW
BETA_2         = 0.999   # Parámetro β₂ de AdamW
EPSILON        = 1e-6    # Epsilon de AdamW para estabilidad numérica

SHUFFLE_BUFFER = 10_000 # Limitamos el tamaño del buffer de shuffle para evitar OOM 

# Forzar el uso de la GPU y activamos el crecimiento de memoria y la limitamo para evitar el OOM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10000)]
    )
    device_name = "GPU"
else:
    device_name = "CPU"

print("Dispositivo:", device_name)
    
print("TensorFlow version:", tf.__version__)
print("Transformers version:", transformers.__version__)

### 1.2 Previsualización de los datos de entrenamiento, validación y test

Aquí he cargado los datasets de manera manual, pensé en hacerlo descargando directamente desde huggingface como hacen en el código de KOTE, pero ya que tenía los archivos quise probar a hacerlo así.

In [ ]:
import pandas as pd
#from datasets import load_dataset

# Cargo los datasets en local pero también podría ser desde HuggingFace como en el notebook que da KOTE: dataset = load_dataset("searle-j/kote")

train_path = "train.tsv"
val_path   = "val.tsv"    
test_path  = "test.tsv"

columns = ["id", "text", "labels"] 
df_train = pd.read_csv(train_path, sep="\t", header=None, names=columns)
df_val   = pd.read_csv(val_path,   sep="\t", header=None, names=columns)
df_test  = pd.read_csv(test_path,  sep="\t", header=None, names=columns)

print(f"Ejemplos cargados de Train: {len(df_train)}, Val: {len(df_val)}, Test: {len(df_test)}")

df_train.head(3)

#### 1.2.1 Control de sesgos de género

In [ ]:
import re

# Definimos el mapeo de términos de género
gender_map = {
    "여자":       "남자",      # mujer -> hombre
    "남자":       "여자",      # hombre -> mujer
    "여성":       "남성",      # femenino -> masculino
    "남성":       "여성",      # masculino -> femenino

    "아버지":     "어머니",    # padre -> madre
    "어머니":     "아버지",    # madre -> padre
    "아들":       "딸",        # hijo -> hija
    "딸":         "아들",      # hija -> hijo
    "남편":       "아내",      # esposo -> esposa
    "아내":       "남편",      # esposa -> esposo
    "오빠":       "언니",      # hermano mayor (hablante femenino) -> hermana mayor
    "언니":       "오빠",      # hermana mayor -> hermano mayor (hablante femenino)
    "형":         "누나",      # hermano mayor (hablante masculino) -> hermana mayor
    "누나":       "형",        # hermana mayor -> hermano mayor (hablante masculino)

    "남자친구":   "여자친구",  # novio -> novia
    "여자친구":   "남자친구",  # novia -> novio
    "총각":       "처녀",      # soltero -> soltera
    "처녀":       "총각",      # soltera -> soltero

    "왕자":       "공주",      # príncipe -> princesa
    "공주":       "왕자",      # princesa -> príncipe
    "왕":         "여왕",      # rey -> reina
    "여왕":       "왕",        # reina -> rey

    "남배우":     "여배우",    # actor -> actriz
    "여배우":     "남배우",    # actriz -> actor

    "그는":       "그녀는",    # él (sujeto) -> ella (sujeto)
    "그녀는":     "그는",      # ella (sujeto) -> él (sujeto)
    "그를":       "그녀를",    # lo/le (objeto) -> la/le (objeto)
    "그녀를":     "그를",      # la/le (objeto) -> lo/le (objeto)
    "그의":       "그녀의",    # su (masculino) -> su (femenino)
    "그녀의":     "그의",      # su (femenino) -> su (masculino)

    "남성적":     "여성적",    # masculino (adjetivo) -> femenino (adjetivo)
    "여성적":     "남성적",    # femenino (adjetivo) -> masculino (adjetivo)
}


# Identificamos las filas cuyos textos contienen alguna clave de gender_map
pattern = "|".join(map(re.escape, gender_map.keys()))
mask = df_train['text'].str.contains(pattern)

# Creamos un DataFrame con las filas a gender-swappear
df_swapped = df_train[mask].copy()

# Aplicamos el reemplazo en la columna de texto
def swap_gender_tokens(txt):
    for src, tgt in gender_map.items():
        txt = txt.replace(src, tgt)
    return txt

df_swapped['text'] = df_swapped['text'].apply(swap_gender_tokens)

# Concatenamos y barajamos el DataFrame resultante antes del split
df_train = pd.concat([df_train, df_swapped], ignore_index=True)
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"Añadidos {len(df_swapped)} ejemplos de género intercambiado. Nuevo tamaño de df_train: {len(df_train)}")

### 1.3 Binarización de las etiquetas

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Para convertir la columna labels de string a lista de ints
def parse_labels(label_str):
    if pd.isna(label_str) or label_str == "":
        return []
    return [int(x) for x in label_str.split(",")]

train_label_lists = df_train["labels"].apply(parse_labels)
val_label_lists   = df_val["labels"].apply(parse_labels)
test_label_lists  = df_test["labels"].apply(parse_labels)

# Pasamos la lisa de etiquetas a un formato multi-hot
# (una lista de listas de etiquetas, donde cada lista tiene el mismo tamaño que el número total de etiquetas)
mlb = MultiLabelBinarizer(classes=list(range(NUM_LABELS)))
mlb.fit(train_label_lists)

y_train = mlb.transform(train_label_lists)
y_val   = mlb.transform(val_label_lists)
y_test  = mlb.transform(test_label_lists)

print("Tamaño de y_train:", y_train.shape)
print("Ejemplo de vector de etiquetas (multi-hot) para una muestra:\n", y_train[0])

### 1.4 Revisión de los comentarios y pasarlos a listas

In [ ]:
# Pasamos los comentarios también a listas
train_texts = df_train["text"].tolist()
val_texts   = df_val["text"].tolist()
test_texts  = df_test["text"].tolist()

print("Texto de ejemplo:", train_texts[0])
print("Etiquetas de ejemplo:", train_label_lists.iloc[0])
print("Vector multi-hot:", y_train[0])

### 1.5 Definición del tokenizador

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

## 2. Definición del modelo 

### 2.1 Carga del modelo preentrenado de transformer

In [ ]:
from transformers import TFAutoModel, AutoConfig

config = AutoConfig.from_pretrained(MODEL_NAME)
transformer_model = TFAutoModel.from_pretrained(MODEL_NAME, config=config)

### 2.2 Pooling de Representaciones y Clasificación

In [ ]:
from tensorflow.keras import Input, Model, regularizers
from tensorflow.keras.layers import Dropout, Dense

def build_model():
    # 1) Carga limpia del transformer
    config = AutoConfig.from_pretrained(MODEL_NAME)
    transformer = TFAutoModel.from_pretrained(MODEL_NAME, config=config)
    # 2) Entradas
    input_ids     = tf.keras.Input(shape=(None,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32, name="attention_mask")
    # 3) Forward + pooling (aquí usamos mean pooling; puedes cambiar a CLS token si lo prefieres)
    outputs = transformer(input_ids=input_ids, attention_mask=attention_mask)
    sequence_output = outputs.last_hidden_state
    pooled_output = tf.reduce_mean(sequence_output, axis=1)
    # 4) Cabeza de clasificación
    x = tf.keras.layers.Dense(256, activation="relu",
                              kernel_regularizer=tf.keras.regularizers.l2(L2_REG)
                             )(pooled_output)
    x = tf.keras.layers.Dropout(DROPOUT_RATE)(x)
    logits = tf.keras.layers.Dense(NUM_LABELS, activation="sigmoid",
                                   kernel_regularizer=tf.keras.regularizers.l2(L2_REG)
                                  )(x)
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=logits)

    return model, transformer

## 3. Entrenamiento

In [ ]:
# ── Función generadora de datos para tf.data.Dataset ──
def data_generator(texts, labels):
    for text, label in zip(texts, labels):
        # Tokeniza sin padding fijo; el padding lo hará el batch
        enc = tokenizer(
            text,
            truncation=True,
            max_length=MAX_LENGTH,
            padding=False
        )
        input_ids     = enc["input_ids"]
        attention_mask = enc["attention_mask"]
        yield (input_ids, attention_mask), label

# ── Tipos y formas de la salida para from_generator ──
output_types = (
    (tf.int32, tf.int32),  # (input_ids, attention_mask)
    tf.int32              # label multietiqueta
)

output_shapes = (
    (tf.TensorShape([None]), tf.TensorShape([None])),  # secuencia variable
    tf.TensorShape([NUM_LABELS])                       # vector de etiquetas
)

In [ ]:
class MicroF1(tf.keras.metrics.Metric):
    def __init__(self, num_labels, threshold=0.5, name="f1_micro", **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_labels = num_labels
        self.threshold  = threshold
        # estados para TP, FP, FN
        self.tp = self.add_weight(name="tp", initializer="zeros")
        self.fp = self.add_weight(name="fp", initializer="zeros")
        self.fn = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        # binariza según threshold
        y_pred = tf.cast(y_pred >= self.threshold, tf.float32)
        y_true = tf.cast(y_true, tf.float32)
        # acumula true positives, false positives, false negatives
        tp = tf.reduce_sum(y_true * y_pred)
        fp = tf.reduce_sum((1 - y_true) * y_pred)
        fn = tf.reduce_sum(y_true * (1 - y_pred))
        self.tp.assign_add(tp)
        self.fp.assign_add(fp)
        self.fn.assign_add(fn)

    def result(self):
        precision = self.tp / (self.tp + self.fp + 1e-7)
        recall    = self.tp / (self.tp + self.fn + 1e-7)
        return 2 * (precision * recall) / (precision + recall + 1e-7)

    def reset_state(self):
        for v in (self.tp, self.fp, self.fn):
            v.assign(0.0)

In [ ]:
from datetime import datetime

run_ts = datetime.now().strftime("%Y%m%d_%H%M%S")
base_ckpt_dir = os.path.join("checkpoints", run_ts)
os.makedirs(base_ckpt_dir, exist_ok=True)

In [ ]:
import psutil
def print_mem(which):
    p = psutil.Process(os.getpid())
    rss = p.memory_info().rss/1024**2
    gpu = tf.config.experimental.get_memory_info("GPU:0")["current"]/1024**2
    print(f"[{which}] RAM = {rss:.1f} MiB, GPU = {gpu:.1f} MiB")

In [ ]:
# ── 1) Combinar train + val ──
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from tensorflow.keras.metrics import AUC
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.mixed_precision import LossScaleOptimizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import re
from tensorflow.data import experimental as tf_exp

texts_all  = train_texts + val_texts
labels_all = np.vstack([y_train, y_val])  # shape = (len(train)+len(val), NUM_LABELS)

# ── 2) Preparar CV ──
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
fold_aucs = []

cache_dir = os.path.expanduser("~/.cache/kote")
os.makedirs(cache_dir, exist_ok=True)

# ── 3) Loop de folds ──
for fold, (train_idx, val_idx) in enumerate(mskf.split(texts_all, labels_all), start=1):
    print_mem(f"fold {fold} — antes clear")
    tf.keras.backend.clear_session() # Limpiamos la sesión de Keras para evitar problemas de memoria
    gc.collect() # Liberamos memoria de objetos no referenciados (recolección de basura)
    print_mem(f"fold {fold} — tras clear & gc")
    print(f"\n>>> Fold {fold}")

    cache_file = os.path.join(cache_dir, f"fold{fold}.tf-data")

    # Partición de textos y etiquetas
    X_tr = [texts_all[i] for i in train_idx]
    y_tr = labels_all[train_idx]
    X_va = [texts_all[i] for i in val_idx]
    y_va = labels_all[val_idx]

    # Construir tf.data.Dataset para este fold
    def seq_len_fn(inputs, labels):
        # inputs es (ids, mask)
        return tf.shape(inputs[0])[0]
    
    train_ds = (
        tf.data.Dataset
        .from_generator(lambda: data_generator(X_tr, y_tr),
                        output_types=output_types,
                        output_shapes=output_shapes)
        .cache(cache_file)   # ← cacheamos la tokenización
        .shuffle(SHUFFLE_BUFFER, seed=SEED)
        .apply(
            tf_exp.bucket_by_sequence_length(
                element_length_func=seq_len_fn,
                bucket_boundaries=[64, 128, 192],
                bucket_batch_sizes=[BATCH_SIZE]*4,
                padded_shapes=(([None], [None]), [NUM_LABELS]),
                padding_values=((tokenizer.pad_token_id, 0), 0),
                drop_remainder=False
            )
        )
        .prefetch(tf.data.AUTOTUNE)
    )

    val_ds = (
        tf.data.Dataset
        .from_generator(lambda: data_generator(X_va, y_va),
                        output_types=output_types,
                        output_shapes=output_shapes)
        .cache()  # ← cacheamos también la validación
        .padded_batch(
            BATCH_SIZE,
            padded_shapes=(([MAX_LENGTH], [MAX_LENGTH]), [NUM_LABELS]),
            padding_values=((tokenizer.pad_token_id, 0), 0)
        )
        .prefetch(tf.data.AUTOTUNE)
    )


    # — Definición de callbacks —
    clean_name = re.sub(r'[^A-Za-z0-9._-]', '_', MODEL_NAME)
    checkpoint_path = os.path.join(base_ckpt_dir, f"fold{fold}_{clean_name}.h5")

    early_stopping_cb = EarlyStopping(monitor="val_f1_micro", mode="max", patience=3, restore_best_weights=True, verbose=1)
    reduce_lr_cb = ReduceLROnPlateau(monitor="val_f1_micro", mode="max", factor=0.5, patience=2, verbose=1)
    checkpoint_cb = ModelCheckpoint(filepath=checkpoint_path, monitor="val_f1_micro", save_best_only=True, mode="max", verbose=1)

    callbacks_fold = [early_stopping_cb, reduce_lr_cb, checkpoint_cb]

    metrics= [
        AUC(name="AUC", multi_label=True),
        MicroF1(num_labels=NUM_LABELS, threshold=0.5, name="f1_micro")
    ] 

    # — Construir modelo desde cero —
    model, transformer_model = build_model()

    # — Pre‐entrenamiento de la cabeza —
    opt1 = AdamW( learning_rate=BASE_LR, weight_decay=WEIGHT_DECAY, beta_1=BETA_1, beta_2=BETA_2, epsilon=EPSILON)
    opt1 = LossScaleOptimizer(opt1)

    transformer_model.trainable = False
    model.compile( optimizer=opt1, loss="binary_crossentropy", metrics= metrics)

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=UNFREEZE_EPOCH-1,
        callbacks=callbacks_fold,
        verbose=1
    )

    # — Fine-tuning completo —
    opt2 = AdamW( learning_rate=FT_LR, weight_decay=WEIGHT_DECAY, beta_1=BETA_1, beta_2=BETA_2, epsilon=EPSILON)
    opt2 = LossScaleOptimizer(opt2)

    transformer_model.trainable = True
    model.compile(optimizer=opt2, loss="binary_crossentropy", metrics=metrics)
    
    model.fit(
        train_ds,
        validation_data=val_ds,
        initial_epoch=UNFREEZE_EPOCH-1,
        epochs=EPOCHS,
        callbacks=callbacks_fold,
        verbose=1
    )

    # Evaluación de este fold
    m = model.evaluate(val_ds, return_dict=True)
    print(f"Fold {fold} — AUC:", m["AUC"])
    fold_aucs.append(m["AUC"])

    # ── Limpieza de objetos del fold ──
    del train_ds, val_ds, model, transformer_model
    gc.collect()
    print_mem(f"fold {fold} — tras delete")


# ── 4) Resultados finales de CV ──
print("\nAUC por fold:", fold_aucs)
print("Media ± std:", np.mean(fold_aucs), "±", np.std(fold_aucs))